# Hidden Trigger Backdoor Attack Example
This notebook shows how to use the ART implementation of the Hidden Trigger Backdoor attack to poison a model. The goal of the attack is to create poisoned inputs within a target class with no visible trigger that can be used during model finetuning to poison the model. After finetuning, adding a trigger to certain inputs not in the target class will cause the model to reliably misclassify those inputs as the target class. Full details of the attack can be found at https://arxiv.org/abs/1910.00033.

In [1]:
import os, sys
from os.path import abspath

module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

import tempfile
# Create a temporary directory for the model checkpoint. Remove the exising one if this cell is rerun
try:
    temp_model_dir.cleanup()
except NameError:
    print("Temporary directory not created yet")
finally:
    temp_model_dir = tempfile.TemporaryDirectory() 
    print("Temporary directory:", temp_model_dir.name)

Temporary directory not created yet
Temporary directory: /tmp/tmp9vc5phno


## Load Dataset
We will load the CIFAR10 dataset as well as define the mean and std as done in the original paper.

In [2]:
import numpy as np
from art.utils import load_dataset
(x_train, y_train), (x_test, y_test), min_, max_ = load_dataset('cifar10')
# Step 1a: Swap axes to PyTorch's NCHW format

x_train = np.transpose(x_train, (0, 3, 1, 2)).astype(np.float32)
x_test = np.transpose(x_test, (0, 3, 1, 2)).astype(np.float32)
mean = (0.4914, 0.4822, 0.4465) 
std = (0.2023, 0.1994, 0.201)

## Define and Train Model
We will use the modified Alexnet model from the original paper as our model to poison. We temporarily save a model checkpoint to disk. This checkpoint will be used during the finetuning step to reset the model to it's original state before the attack. For some reason, some model state variable seems to change after the attack is performed. 

In [3]:
from art.estimators.classification import PyTorchClassifier
import torch
import torch.nn as nn
import torch.optim as optim

num_classes=10
feature_size=4096
model=nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.Conv2d(64, 192, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.Conv2d(192, 384, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.Flatten(),
        nn.Dropout(),
        nn.Linear(256 * 1 * 1, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(),
        nn.Linear(4096, feature_size),
        nn.ReLU(inplace=True),
        nn.Linear(feature_size, num_classes)
)

# Define the ART Estimator
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=2e-4)
classifier = PyTorchClassifier(
    model=model,
    clip_values=(min_, max_),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 32, 32),
    nb_classes=10,
    preprocessing=(mean, std)
)

In [4]:
# Train the model

classifier.fit(x_train, y_train, nb_epochs=100, batch_size=128, verbose=True)
for param_group in classifier.optimizer.param_groups:
    print(param_group["lr"])
    param_group["lr"] *= 0.1
classifier.fit(x_train, y_train, nb_epochs=50, batch_size=128, verbose=True)
for param_group in classifier.optimizer.param_groups:
    print(param_group["lr"])
    param_group["lr"] *= 0.1
classifier.fit(x_train, y_train, nb_epochs=50, batch_size=128, verbose=True)
torch.save(model.state_dict(), temp_model_dir.name + "/htbd_model.pth") # Write the checkpoint to a temporary directory



0.01
0.001


In [5]:
predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

Accuracy on benign test examples: 77.19%


## Generate the Poison
We now will generate the poison using the hidden trigger backdoor attack. First, we define the target and source classes as well as the backdoor trigger. The target class will be the class we want to insert poisoned data into. The source class will be the class we will add a trigger to in order to cause misclassification into the target.

The backdoor trigger will be a small image patch inserted into the source class images. At test time, we should be able to use this trigger to cause the classifier to misclassify source class images with the trigger added as the target class.

In [6]:
from art.attacks.poisoning.backdoor_attack import PoisoningAttackBackdoor
target = np.array([0,0,0,0,1,0,0,0,0,0])
source = np.array([0,0,0,1,0,0,0,0,0,0])

# Backdoor Trigger Parameters
patch_size = 8
x_shift = 32 - patch_size - 5
y_shift = 32 - patch_size - 5

# Define the backdoor poisoning object. Calling backdoor.poison(x) will insert the trigger into x.
from art.attacks.poisoning import perturbations
def mod(x):
    original_dtype = x.dtype
    x = perturbations.insert_image(x, backdoor_path="../../utils/data/backdoors/htbd.png",
                                   channels_first=True, random=False, x_shift=x_shift, y_shift=y_shift,
                                   size=(patch_size,patch_size), mode='RGB', blend=1)
    return x.astype(original_dtype)
backdoor = PoisoningAttackBackdoor(mod)

Here we run the attack. `eps` controls how much the target images can be perturbed with respect to an l-infinity distance. `feature_layer` dicates with layer's output will be used to define the attack's loss. It can either be the name of the layer or the layer index according to the ART estimator. `poison_percent` controls how many poisoned samples will be generated based on the size of the input data.

The attack will return poisoned inputs of the target class and the indicies in the data that those poisoned inputs should replace.

In [7]:
from art.attacks.poisoning import HiddenTriggerBackdoor
poison_attack = HiddenTriggerBackdoor(classifier, eps=16/255, target=target, source=source, feature_layer=19, backdoor=backdoor, decay_coeff = .95, decay_iter = 2000, max_iter=5000, batch_size=25, poison_percent=.01)

poison_data, poison_indices = poison_attack.poison(x_train, y_train)
print("Number of poison samples generated:", len(poison_data))

Hidden Trigger:   0%|          | 0/2 [00:00<?, ?it/s]

Epoch:  0 | batch: 0 | i:     0 | LR: 0.00100 |                         Loss Val: 3277.405 | Loss Avg: 3277.405
Epoch:  0 | batch: 0 | i:   100 | LR: 0.00100 |                         Loss Val: 13.257 | Loss Avg: 128.395
Max_Loss: 9.954315185546875
Epoch:  0 | batch: 1 | i:     0 | LR: 0.00100 |                         Loss Val: 3344.732 | Loss Avg: 124.482
Max_Loss: 9.848298072814941
Number of poison samples generated: 50


## Finetune the Model
Now, we must finetune the model using the poisoned data and a small number of clean training inputs.  Here, we randomly select an equal number of training inputs from each of the classes.

In [8]:
# Create finetuning dataset
dataset_size = 2500
num_classes = 10
num_per_class = dataset_size/num_classes

poison_dataset_inds = []

for i in range(num_classes):
    class_inds = np.where(np.argmax(y_train,axis=1) == i)[0]
    num_select = int(num_per_class)
    if np.argmax(target) == i:
        num_select = int(num_select - len(poison_data))
        poison_dataset_inds.append(poison_indices)
    poison_dataset_inds.append(np.random.choice(class_inds, num_select, replace=False))
    
poison_dataset_inds = np.concatenate(poison_dataset_inds)

poison_x = np.copy(x_train)
poison_x[poison_indices] = poison_data
poison_x = poison_x[poison_dataset_inds]

poison_y = np.copy(y_train)[poison_dataset_inds]

We reload the model from the temporary checkpoint created before to reset the model to its original state after training just in case something changed from the attack. We freeze all of the layers and then replace the final layer with an randomly initialized layer.

In [9]:
import torch.nn as nn

# Load model again
num_classes=10
feature_size=4096
model=nn.Sequential(
        nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.Conv2d(64, 192, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.Conv2d(192, 384, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(384, 256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True),
        nn.Conv2d(256, 256, kernel_size=3, stride=1, padding=1),
        nn.ReLU(inplace=True),
        nn.MaxPool2d(kernel_size=3, stride=2),
        nn.Flatten(),
        nn.Dropout(),
        nn.Linear(256 * 1 * 1, 4096),
        nn.ReLU(inplace=True),
        nn.Dropout(),
        nn.Linear(4096, feature_size),
        nn.ReLU(inplace=True),
        nn.Linear(feature_size, num_classes)
)
model.load_state_dict(torch.load(temp_model_dir.name+"/htbd_model.pth"))
temp_model_dir.cleanup() # Remove the temporary directory after loading the checkpoint

# Freeze the layers up to the last layer
for i, param in enumerate(model.parameters()):
    param.requires_grad = False


num_classes=10
feature_size=4096
model[20] = nn.Linear(feature_size, num_classes)


criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.5, momentum=0.9, weight_decay=2e-4)

classifier = PyTorchClassifier(
    model=model,
    clip_values=(min_, max_),
    loss=criterion,
    optimizer=optimizer,
    input_shape=(3, 32, 32),
    nb_classes=10,
    preprocessing=(mean, std)
)

In [10]:
trigger_test_inds = np.where(np.all(y_test == source, axis=1))[0]

lr_factor = .1
lr_schedule = [5, 10, 15]

test_poisoned_samples, test_poisoned_labels  = backdoor.poison(x_test[trigger_test_inds], y_test[trigger_test_inds])

for i in range(4):
    print("Training Epoch", i*5)
    predictions = classifier.predict(x_test)
    accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
    print("Accuracy on benign test examples: {}%".format(accuracy * 100))
    
    predictions = classifier.predict(x_test[trigger_test_inds])
    b_accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[trigger_test_inds], axis=1)) / len(trigger_test_inds)
    print("Accuracy on benign trigger test examples: {}%".format(b_accuracy * 100))
    
    predictions = classifier.predict(test_poisoned_samples)
    p_accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(test_poisoned_labels,axis=1)) / len(test_poisoned_labels)
    print("Accuracy on poison trigger test examples: {}%".format(p_accuracy * 100))
    p_success = np.sum(np.argmax(predictions, axis=1) == np.argmax(target)) / len(test_poisoned_labels)
    print("Success on poison trigger test examples: {}%".format(p_success * 100))
    print()
    if i != 0:
        for param_group in classifier.optimizer.param_groups:
            param_group["lr"] *= lr_factor
    classifier.fit(poison_x, poison_y, epochs=5, training_mode=False)


Training Epoch 0
Accuracy on benign test examples: 11.82%
Accuracy on benign trigger test examples: 0.0%
Accuracy on poison trigger test examples: 0.0%
Success on poison trigger test examples: 0.4%

Training Epoch 5
Accuracy on benign test examples: 73.19%
Accuracy on benign trigger test examples: 44.4%
Accuracy on poison trigger test examples: 24.0%
Success on poison trigger test examples: 62.2%

Training Epoch 10
Accuracy on benign test examples: 72.17%
Accuracy on benign trigger test examples: 38.3%
Accuracy on poison trigger test examples: 15.7%
Success on poison trigger test examples: 73.6%

Training Epoch 15
Accuracy on benign test examples: 72.35000000000001%
Accuracy on benign trigger test examples: 39.6%
Accuracy on poison trigger test examples: 17.299999999999997%
Success on poison trigger test examples: 71.89999999999999%



In [11]:
print("Final Performance")
predictions = classifier.predict(x_test)
accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test, axis=1)) / len(y_test)
print("Accuracy on benign test examples: {}%".format(accuracy * 100))

predictions = classifier.predict(x_test[trigger_test_inds])
b_accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[trigger_test_inds], axis=1)) / len(trigger_test_inds)
print("Accuracy on benign trigger test examples: {}%".format(b_accuracy * 100))

predictions = classifier.predict(test_poisoned_samples)
p_accuracy = np.sum(np.argmax(predictions, axis=1) == np.argmax(y_test[trigger_test_inds],axis=1)) / len(trigger_test_inds)
print("Accuracy on poison trigger test examples: {}%".format(p_accuracy * 100))
p_success = np.sum(np.argmax(predictions, axis=1) == np.argmax(target)) / len(trigger_test_inds)
print("Success on poison trigger test examples: {}%".format(p_success * 100))

Final Performance
Accuracy on benign test examples: 72.36%
Accuracy on benign trigger test examples: 39.6%
Accuracy on poison trigger test examples: 17.299999999999997%
Success on poison trigger test examples: 71.8%
